# 1. 데이터 불러오기

In [150]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

df=pd.read_csv('casting.csv',encoding='cp949',index_col=0)

C:\Users\USER\AppData\Local\Temp\ipykernel_31168\3959905438.py:12: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('casting.csv',encoding='cp949',index_col=0)


In [151]:

def make_df(df):
    """전처리 함수"""
    
    # 결측치 처리
    df['heating_furnace']=df['heating_furnace'].fillna('C')
    df['molten_temp']=df['molten_temp'].fillna(df['molten_temp'].mean())
    df['upper_mold_temp3']=df['upper_mold_temp3'].fillna(method='ffill')
    df['lower_mold_temp3']=df['lower_mold_temp3'].fillna(method='bfill')
    
    # 필요없는 열 제거
    df.drop(['line','name','mold_name','emergency_stop','tryshot_signal','time','date','molten_volume','registration_time'],axis=1,inplace=True)
    
    # 결측치 날리기
    df=df.dropna(subset=['count', 'working', 'molten_temp',
       'facility_operation_cycleTime', 'production_cycletime',
       'low_section_speed', 'high_section_speed',
       'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
       'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1',
       'lower_mold_temp2', 'lower_mold_temp3', 'sleeve_temperature',
       'physical_strength', 'Coolant_temperature', 'EMS_operation_time',
        'passorfail', 'mold_code'])
    
    # 이상치 처리할 열 지정
    column=['molten_temp', 'facility_operation_cycleTime',
           'production_cycletime', 'low_section_speed', 'high_section_speed',
           'cast_pressure', 'biscuit_thickness',
           'upper_mold_temp1', 'upper_mold_temp2', 'upper_mold_temp3',
           'lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3',
           'sleeve_temperature', 'physical_strength', 'Coolant_temperature',
           'EMS_operation_time']
    
    # 이상치 처리 (3시그마 벗어나는 범위는 평균으로 대체)
    for i in column:
        std=df[i].std()*3
        mean=df[i].mean()
    
        min_out=mean-std
        max_out=mean+std
    
        df.loc[df[i]<min_out,i]=mean
        df.loc[df[i]>max_out,i]=mean
        
    return df

In [152]:
# 전처리
df['heating_furnace']=df['heating_furnace'].fillna('C')
df['molten_temp']=df['molten_temp'].fillna(df['molten_temp'].mean())
df['upper_mold_temp3']=df['upper_mold_temp3'].fillna(method='ffill')
df['lower_mold_temp3']=df['lower_mold_temp3'].fillna(method='bfill')

#  필요없는 열 제거
df.drop(['line','name','mold_name','emergency_stop','tryshot_signal','time','date','molten_volume','registration_time'],axis=1,inplace=True)

# 결측치 날리기
df=df.dropna(subset=['count', 'working', 'molten_temp',
       'facility_operation_cycleTime', 'production_cycletime',
       'low_section_speed', 'high_section_speed',
       'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
       'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1',
       'lower_mold_temp2', 'lower_mold_temp3', 'sleeve_temperature',
       'physical_strength', 'Coolant_temperature', 'EMS_operation_time',
        'passorfail', 'mold_code'])

# 이상치 처리할 열 지정
column=['molten_temp', 'facility_operation_cycleTime',
           'production_cycletime', 'low_section_speed', 'high_section_speed',
           'cast_pressure', 'biscuit_thickness',
           'upper_mold_temp1', 'upper_mold_temp2', 'upper_mold_temp3',
           'lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3',
           'sleeve_temperature', 'physical_strength', 'Coolant_temperature',
           'EMS_operation_time']

# 이상치 처리 (3시그마 벗어나는 범위는 평균으로 대체)
for i in column:
    std=df[i].std()*3
    mean=df[i].mean()
    
    min_out=mean-std
    max_out=mean+std
    
    df.loc[df[i]<min_out,i]=mean
    df.loc[df[i]>max_out,i]=mean
    
# 범주형 원핫인코딩하기
df=pd.get_dummies(df, columns=['working', 'heating_furnace','mold_code'])

# 표준화하기
minmax_scaler=MinMaxScaler()
select_column=['count', 'molten_temp', 'facility_operation_cycleTime',
       'production_cycletime', 'low_section_speed', 'high_section_speed',
       'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
       'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1',
       'lower_mold_temp2', 'lower_mold_temp3', 'sleeve_temperature',
       'physical_strength', 'Coolant_temperature', 'EMS_operation_time',
       'passorfail']
maxmax_scaled=minmax_scaler.fit_transform(df[select_column])

df[select_column]=maxmax_scaled

# train, val데이터 분리
X=df.drop(['passorfail'],axis=1)
y=df['passorfail']

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=2021)

# 랜덤포레스트 모델
#RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_train,y_train)
pred=model.predict_proba(X_val)
# 성능확인
roc_auc_score(y_val, pred[:,1])


0.9973051539487486

## 모델 선택 및 결과 출력

In [ ]:
model=RandomForestClassifier()
model.fit(X_train,y_train)
pred = model.predict(X_val)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
submission = pd.DataFrame({
        "ID": X_val.index,
        "passorfail": pred
    })

In [ ]:
submission.head()

,ID,passorfail
0,38699,0.0
1,6144,0.0
2,10800,0.0
3,42049,0.0
4,60061,0.0


## 3. 모델링

### 3-1. 타겟값 빈도 확인하기('passorfail')

In [148]:
df['passorfail'].value_counts()

0.0    87998
1.0     4016
Name: passorfail, dtype: int64

In [108]:
#LogisticRegression
model=LogisticRegression()
model.fit(X_train,y_train)
pred=model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.954667048108555

In [109]:
#KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train,y_train)
pred=model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

0.9516842221936967

In [110]:
#DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(X_train,y_train)
pred=model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])


0.9273970563259114

In [111]:
#RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_train,y_train)
pred=model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

0.9981210482174966

In [112]:
#XGBClassifier
model = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False)
model.fit(X_train,y_train)
pred=model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

0.996994465733828

## 모델 선택 및 결과 출력

In [121]:
model=RandomForestClassifier()
model.fit(X_train,y_train)
pred = model.predict(X_val)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [122]:
submission = pd.DataFrame({
        "ID": X_val.index,
        "passorfail": pred
    })

In [124]:
submission.head()

,ID,passorfail
0,38699,0.0
1,6144,0.0
2,10800,0.0
3,42049,0.0
4,60061,0.0
